In this notebook we experiment with analyzing all the github data for 2015 using [dask](http://dask.pydata.org/en/latest/) ([github](https://github.com/continuumIO/dask)) for analysis.


**Dask** is a tool for out-of-core, parallel data analysis. We will use [dask.bag](http://dask.pydata.org/en/latest/bag.html), which provides an api for operations on unordered lists (like sets but with duplicates). It is useful for semi-structured data like JSON blobs or log files. More blogposts about dask can be found [here](http://www.continuum.io/blog/tags/dask) or [here](http://matthewrocklin.com/blog/tags.html#dask-ref).

### Github Archive Data on Google Cloud Storage

We took data from [githubarchive.com](githubarchive.com), from January 2015 and put it in Google Cloud Storage so we can get free transfers between there, and google compute, which runs binder.

Lets inspect the data first so we can find something to analyze and learn the data schema.

In [ ]:
!rm github-data.tar
!wget  https://storage.googleapis.com/blaze-data/github-data/github-data.tar

In [ ]:
!tar xf github-data.tar

In [ ]:
!du -h data/  # 4.6GB

### Inspect Data with `dask.bag`

We have approximately 4.6 GB of data. One file per hour, averaging around 7.8 MB each (compressed). So we make a dask bag with the data and inspect it to figure out the schema.

In [ ]:
from dask.diagnostics import ProgressBar
import dask.bag as db
import ujson as json

# take one file from the bucket load it as a json object, not gz decompression
# happens automatically at compute time.
b = db.from_filenames('data/2015-*.json.gz').map(json.loads)
b.npartitions  # number of files

In [ ]:
first = b.take(1)[0]  # take the first json object from the file
first

In [ ]:
first.keys()  # top level keys in this json object

Type looks interesting. What are possible types and how often does each occur? We can inspect this with `dask.bag.frequencies`. But this takes longer because it requires a read of the entire dataset.

In [ ]:
with ProgressBar():
    res = b.pluck('type').frequencies().compute()
res

Top Committers
----------------

So most events are pushes, that is not surprising. Lets ask "Who pushes the most?".

We do this by filtering out `PushEvent`s. Then we count the frequencies of usernames for the pushes. Then take the top 5.

In [ ]:
pushes = b.filter(lambda x: x['type'] == 'PushEvent')  # filter out the push events
names = pushes.pluck('actor').pluck('login') # get the login names
top_5 = names.frequencies().topk(5, key=lambda (name, count): count)  # List top 5 pushers
with ProgressBar():
    res = top_5.compute()  # run the above computations
res

These users *pushed* the most, but push can have multiple commits. So we can ask "who pushed the most *commits*?".

We can figure this out by grouping by username, then summing the number of commits from every push, for each user. More technically speaking, we want to `GroupBy` on usernames, so for each username we get a list their of PushEvents. Then reduce each `PushEvent` by taking a `count` of their commits. Then reducing these `count`s by `sum`ing them for each user. So we are grouping then reducing.

However there are algorithms for grouping and reducing simultaneously which avoid expensive shuffle operations and are much faster. In dask bag we have `foldby`. Analogous methods: [`toolz.reduceby`]( http://toolz.readthedocs.org/en/latest/api.html#toolz.itertoolz.reduceby), and in pyspark [`RDD.combineByKey`](https://spark.apache.org/docs/latest/api/python/pyspark.html?#pyspark.RDD.combineByKey).

In [ ]:
def get_logins(x):
    """The key for foldby, like a groupby key. Get the username from a PushEvent"""
    return x['actor']['login']

def binop(total, x):
    """Count the number of commits in a PushEvent"""
    return total + len(x['payload']['commits'])

def combine(total1, total2):
    """This combines commit counts from PushEvents"""
    return total1 + total2

commits = pushes.foldby(get_logins, binop, initial=0, combine=combine)
top_commits = commits.topk(5, key=lambda (name, count): count)
with ProgressBar():
    res = top_commits.compute()
res